# Langchain

Langchain is a bullshit framework for developing LLM applications.

It provides no real value over directly using LLMs API.

It adds a crazy amount of abstraction (2001 java-like amount) for what essentially is function composition, f-strings, and collection of prompts.

Code becomes hard to debug (and they sell debugging tool called LangSmith), everything is indirect.

In theory, langchain makes switching between different LLMs easy, but good luck with that.

In [38]:
import langchain

In [39]:
import getpass
import os

os.environ['OPENAI_API_KEY'] = getpass.getpass(
    'enter openai api key, you can get it from https://platform.openai.com/api-keys')



In [40]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o')

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="answer the questions as if you're Eminem"),
    HumanMessage(content="how to reverse a string in Python?"),
]

response = model.invoke(messages)
print(response)

content='Sure, reversing a string in Python is pretty simple. You can use slicing to achieve that. Here\'s how you do it:\n\n```python\noriginal_string = "your_string_here"\nreversed_string = original_string[::-1]\nprint(reversed_string)\n```\n\nSo if you had, like, a string "Eminem", and you wanted to reverse it, it\'d go something like this:\n\n```python\noriginal_string = "Eminem"\nreversed_string = original_string[::-1]\nprint(reversed_string)  # Output will be "menimE"\n```\n\nThat\'s it. Quick and easy, just like dropping bars.' response_metadata={'token_usage': {'completion_tokens': 128, 'prompt_tokens': 27, 'total_tokens': 155}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None} id='run-446cb159-405a-4b97-bbe9-cc268fe29ead-0' usage_metadata={'input_tokens': 27, 'output_tokens': 128, 'total_tokens': 155}


`model.invoke` returns an instance of `AIMessage`
with `StrOutputParser` we can convert it to string

In [41]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(response)

'Sure, reversing a string in Python is pretty simple. You can use slicing to achieve that. Here\'s how you do it:\n\n```python\noriginal_string = "your_string_here"\nreversed_string = original_string[::-1]\nprint(reversed_string)\n```\n\nSo if you had, like, a string "Eminem", and you wanted to reverse it, it\'d go something like this:\n\n```python\noriginal_string = "Eminem"\nreversed_string = original_string[::-1]\nprint(reversed_string)  # Output will be "menimE"\n```\n\nThat\'s it. Quick and easy, just like dropping bars.'

Or we can create a chain by using `|` operator

In [42]:
chain = model | parser

In [43]:
chain.invoke(messages)

'Alright, so you wanna reverse a string in Python, huh? Here\'s how you do it:\n\n```python\nyour_string = "reverse this"\nreversed_string = your_string[::-1]\nprint(reversed_string)\n```\n\nThat slice notation `[::-1]` is like flipping the script on the whole string, taking it from end to start. Easy as that.'

We can create `messages` from a template

In [44]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "answer the questions as if you're {character}"
user_template = "{text}"

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template), ("user", user_template)])

Now we can use `prompt_template` to create prompts

In [45]:
prompt_value = prompt_template.invoke(
    {
        "character": "Taylor Swift",
        "text": "how to reverse a string in Python?"
    })
prompt_value

ChatPromptValue(messages=[SystemMessage(content="answer the questions as if you're Taylor Swift"), HumanMessage(content='how to reverse a string in Python?')])

As you can see it's just a simple string substitution and list generation

In [46]:
chain = prompt_template | model | parser

In [47]:
chain.invoke(
    {
        "character": "Taylor Swift",
        "text": "how to reverse a string in Python?"
    })

'Sure, I can help with that! Reversing a string in Python can be done in several ways, but one of the simplest is using slicing. Here’s a quick example:\n\n```python\noriginal_string = "Hello, World!"\nreversed_string = original_string[::-1]\nprint(reversed_string)\n```\n\nIn this example, `[::-1]` is a slice that tells Python to take the string and step backward by one, effectively reversing it. It’s a neat little trick! Hope that helps!'

### Langserve
See [langchain_with_langserve.py](./langchain_with_langserve.py) for a langchain-powered server

You can run it with OPENAI_API_KEY=xxx python langchain_with_langserve.py
You can play with you model on http://localhost:8000/chain/playground with UI.

Or you can access it programmatically:

In [48]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain")
remote_chain.invoke({"character": "Guido van Rossum", "text": "How to reverse string in Python?"})

'You can reverse a string in Python by using slicing. Here\'s an example:\n\n```python\noriginal_string = "hello"\nreversed_string = original_string[::-1]\nprint(reversed_string)\n```\n\nThis will output: "olleh"'

### React

ReACT is a way to implement "intelligent" agent.
You give a prompt to LLM like 'You have access to a tool that can frobnicate. When you need to frobnicate to give a better answer you should output json with {"tool": "frobnicate", "args": ["x"]}'. When you get this json from LLM you just call some code. In langchain this is done with tools

In [49]:
from langchain.tools import tool


@tool
def frobnicate(something: str) -> str:
    """
    Frobnicate something.
    """
    return f"langchain provides very little value over {something}"

Let's ask a model to frobnicate without tool:

In [51]:
print(chain.invoke({"character": "Batman", "text": "What's the result of frobnification of 'openai python api'?"}))

The term "frobnification" isn't standard in any of my known databases or in common cryptographic or computational practices. However, it sounds like a whimsical or placeholder term for some form of transformation or encoding process. 

Without specifics, it's hard to determine the exact outcome. If you've got more details about the algorithm or method behind this "frobnification," I could give you a more precise answer. Until then, stay vigilant, and always question the unknown.

I'm Batman.


The model can't frobnicate out of the box. Let's teach it do frobnicate. 

In [56]:
model_with_tools = model.bind_tools([frobnicate])
chain_with_tools = prompt_template | model_with_tools

In [69]:
response = chain_with_tools.invoke(
    {"character": "Batman", "text": "What's the uppercase result of frobnification of 'openai python api'?"})

In [71]:
print(f"{response.tool_calls=}")
print(f"{response.content=}")

response.tool_calls=[{'name': 'frobnicate', 'args': {'something': 'openai python api'}, 'id': 'call_WlWMeh774WeWeOdlvMloL5EF'}]
response.content=''


`response.tool_calls` were actually not called (notice that the content is empty), we need to create an agent to use tools

In [62]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(model_with_tools, [frobnicate])

In [73]:
response = agent.invoke({"messages": "What's the uppercase result of frobnification of 'openai python api'?"})

In [74]:
print(response["messages"][-1].content)

The result of the frobnication of "openai python api" is "langchain provides very little value over openai python api". 

The uppercase version of this result is:

"LANGCHAIN PROVIDES VERY LITTLE VALUE OVER OPENAI PYTHON API"


Absolutely correct! This is called Singularity.

### RAG

RAG (retrieval-augmented generation) allows you to add some extra context to LLM.
You have your own private data, that LLM was not trained on.
So you can take user query, find part of the documents that are relevant to the query
and add this information as part of the context to LLM.

So it gives an illusion as if LLM "knows" about some of your private data.  

Finding relevant documents is usually done with vector DBs by converting text to vectors and calculating e.g. cosine similarity between two vectors

In [84]:
# Chroma is a vector DB
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough

docs = [
    Document('My favorite color is blue'),
    Document('My favorite movies are Glengarry Glen Ross and Godfather Part 2'),
]

vectorstore = Chroma.from_documents(docs, embedding=OpenAIEmbeddings())


def format_docs(docs):
    return "\n\n".join(a_doc.page_content for a_doc in docs)


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)

# return single most relevant document
golden_retriever = vectorstore.as_retriever(search_kwargs={'k': 1})

after_prompt = {
    "context": golden_retriever | format_docs,
    "question": RunnablePassthrough()
} | prompt

rag_chain =  after_prompt | model | StrOutputParser()


In [85]:
resp = rag_chain.invoke("what is my favorite color?")

In [86]:
print(resp)

Your favorite color is blue.


Here's a proof that we give LLM only one relevant document in `context`:

In [90]:
print(after_prompt.invoke("what is my favorite color?").messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.

Context: My favorite color is blue


In [93]:
print(rag_chain.invoke("what actor stars in my favorite movies?"))

Al Pacino stars in both "Glengarry Glen Ross" and "The Godfather Part II."


Here's a proof that we give LLM only one relevant document in a `context`:

In [95]:
print(after_prompt.invoke("what actor stars in my favorite movies?").messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.

Context: My favorite movies are Glengarry Glen Ross and Godfather Part 2
